<a href="https://colab.research.google.com/github/Samgomes2510/Projeto_Final_Dashboard_de_Ecommerce_SQL/blob/main/Projeto_Final_dados_ecommerce_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MÓDULO 26 - Projeto Final do Aprofundamento de Analytics**
---------------------------------------------------------------
Objetivo: Criar um Dashboard de E-commerce consolidando dados de transações e clientes
Ferramentas: Python, SQLite, Power BI / Looker Studio


In [1]:
# Importando bibliotecas essenciais
import sqlite3  # Para criar e manipular o banco de dados SQLite
import pandas as pd  # Para manipulação de dados em DataFrames


**1️⃣ Leitura das tabelas de dados**
Carregaremos as tabelas CSV de transações e clientes

In [2]:
# Leitura dos arquivos CSV (substitua pelo caminho correto se estiver localmente)
df_transacoes = pd.read_csv("TB_TRANSACOES_PROJETO_ECOMM.csv", delimiter=';')
df_clientes = pd.read_csv("TB_CLIENTES_PROJETO_ECOMM.csv", delimiter=';')

In [3]:
# Exibir as primeiras linhas de cada base
print("Tabela de Transações:")
display(df_transacoes.head())
print("\nTabela de Clientes:")
display(df_clientes.head())

Tabela de Transações:


,id_client,Category,Price,Card Type
0,37,Electronics,"72,93",mastercard
1,38,Jewelry,"121,89",mastercard
2,39,Baby,"64,3",mastercard
3,40,Outdoors,"9,48",mastercard
4,5,Outdoors,"61,95",mastercard



Tabela de Clientes:


,state_name,First_name,Gender,Job_Title,Id_client
0,TX,Domingo,Male,Structural Analysis Engineer,1
1,MI,Russell,Male,Speech Pathologist,2
2,AL,Kimble,Male,Account Coordinator,3
3,IL,Barnabas,Male,General Manager,4
4,MN,Tanney,Female,VP Marketing,5


**2️⃣ Criação do Banco de Dados SQLite e carregamento das tabelas**



In [4]:
# Criando o banco SQLite local
conn = sqlite3.connect('projeto_ecommerce.db')

# Gravando as tabelas no banco de dados
df_transacoes.to_sql('tb_transacoes', conn, index=False, if_exists='replace')
df_clientes.to_sql('tb_clientes', conn, index=False, if_exists='replace')

print("✅ Tabelas salvas com sucesso no banco SQLite!")


✅ Tabelas salvas com sucesso no banco SQLite!


 **3️⃣ Função auxiliar para executar queries SQL e retornar DataFrames**



In [5]:
def run_query(query):
    """Executa uma consulta SQL e retorna um DataFrame com os resultados."""
    return pd.read_sql_query(query, conn)



 **4️⃣ Consulta SQL: Unindo as tabelas**

In [12]:
# Vamos unir as tabelas tb_transacoes e tb_clientes usando a chave ID_CLIENT.

query = """
SELECT
    t.*, -- Select all columns from tb_transacoes
    c.First_name,
    c.Gender,
    c.state_name -- Select specified columns from tb_clientes
FROM tb_transacoes AS t
LEFT JOIN tb_clientes AS c
ON t.id_client = c.Id_client
"""

In [13]:
# Execute the query
df_final = run_query(query)

# Display the first few rows of the resulting DataFrame
display(df_final.head())

,id_client,Category,Price,Card Type,First_name,Gender,state_name
0,37,Electronics,"72,93",mastercard,Cornie,Genderqueer,ND
1,38,Jewelry,"121,89",mastercard,Rab,Male,PA
2,39,Baby,"64,3",mastercard,Codie,Female,MA
3,40,Outdoors,"9,48",mastercard,Scott,Female,OR
4,5,Outdoors,"61,95",mastercard,Tanney,Female,MN


In [14]:
# Exibir amostra dos dados unidos
print("✅ JOIN realizado com sucesso! Exibindo amostra dos dados:")
display(df_final.head())



✅ JOIN realizado com sucesso! Exibindo amostra dos dados:


,id_client,Category,Price,Card Type,First_name,Gender,state_name
0,37,Electronics,"72,93",mastercard,Cornie,Genderqueer,ND
1,38,Jewelry,"121,89",mastercard,Rab,Male,PA
2,39,Baby,"64,3",mastercard,Codie,Female,MA
3,40,Outdoors,"9,48",mastercard,Scott,Female,OR
4,5,Outdoors,"61,95",mastercard,Tanney,Female,MN


 **5️⃣ Justificativa do JOIN**

Foi utilizado o *LEFT JOIN* porque desejamos manter todas as transações do e-commerce,
mesmo que algum cliente tenha informações faltantes na tabela de clientes.
O foco é garantir que nenhuma venda seja perdida na análise.



 **6️⃣ Exportando os dados unificados para CSV**



In [15]:
# Exportando o DataFrame final para CSV
df_final.to_csv('dados_ecommerce_final.csv', index=False, encoding='utf-8-sig')
print("✅ Arquivo 'dados_ecommerce_final.csv' exportado com sucesso!")





✅ Arquivo 'dados_ecommerce_final.csv' exportado com sucesso!


**7️⃣ Próximos passos no Power BI / Looker Studio**
- Importar o arquivo dados_ecommerce_final.csv
- Criar visualizações como:
- Total de vendas por categoria
- Vendas por cidade e estado
- Perfil demográfico dos clientes (gênero, faixa etária)
- Ticket médio
- Quantidade de transações por período
 Utilizar filtros interativos e segmentadores



 **8️⃣ Exploração rápida em Python**


In [17]:
# Estatísticas rápidas de vendas
print("📈 Estatísticas de Valor de Transação:")
display(df_final['Price'].describe())

📈 Estatísticas de Valor de Transação:


,Price
count,367
unique,362
top,"39,03"
freq,2


In [19]:
# Top 5 cidades com maior volume de vendas
print("\n🏙️ Top 5 cidades/estados com maior volume de vendas:")
display(df_final.groupby('state_name')['Price'].sum().sort_values(ascending=False).head())


🏙️ Top 5 cidades/estados com maior volume de vendas:


,Price
state_name,
SD,"99,37142,25"
KS,"99,16101,21135,37112,8946,16"
TX,"97,3892,52108,425,4569,98123,3859,74100,6296,6..."
NC,"96,1639,52134,09104,631,8656,68129,51"
CT,"95,54119,7394,58"


In [21]:
# Distribuição de vendas por categoria de produto
print("\n🛍️ Distribuição de vendas por categoria:")
display(df_final['Category'].value_counts())


🛍️ Distribuição de vendas por categoria:


,count
Category,
Kids,27
Beauty,23
Industrial,22
Jewelry,22
Tools,20
Shoes,19
Music,19
Computers,18
Grocery,18
